In [ ]:
import easyocr
import cv2
import numpy as np
import re

# Initialize the OCR reader
reader = easyocr.Reader(lang_list=['en'], gpu=True)

# Function to extract text from an image
def extract_text_from_image(image):
    try:
        results = reader.readtext(image)
    except Exception as e:
        print(f"An error occurred while reading the text: {e}")
        results = []

    # Extracted text list
    extracted_text = [result[1] for result in results]
    return extracted_text

# Function to summarize extracted information
def summarize_information(extracted_text):
    summary = {
        "Product Name": None,
        "Brand Name": None,
        "Expiry Date": None,
        "Manufacturing Date": None,
        "MRP": None,
        "Price": None,
        "Quantity": None
    }

    # Combine the extracted text into a single string for easier searching
    combined_text = ' '.join(extracted_text)

    # Use regex to find patterns for each field
    product_name_match = re.search(r'(?:Product\s*Name:?)\s*([A-Za-z0-9\s]+)', combined_text)
    brand_name_match = re.search(r'(?:Brand\s*Name:?)\s*([A-Za-z0-9\s]+)', combined_text)
    expiry_date_match = re.search(r'(?:Expiry\s*Date:?)\s*([0-9]{2}/[0-9]{2}/[0-9]{4})', combined_text)
    manufacturing_date_match = re.search(r'(?:Manufacturing\s*Date:?)\s*([0-9]{2}/[0-9]{2}/[0-9]{4})', combined_text)
    mrp_match = re.search(r'(?:MRP:?)\s*\₹?([0-9]+(?:\.[0-9]{1,2})?)', combined_text)
    price_match = re.search(r'(?:Price:?)\s*\₹?([0-9]+(?:\.[0-9]{1,2})?)', combined_text)
    quantity_match = re.search(r'(?:Quantity\s*|Qty\s*|Count\s*|No\s*|Number\s*):?\s*([0-9]+)', combined_text)

    # Assign values to summary if matches are found
    if product_name_match:
        summary["Product Name"] = product_name_match.group(1).strip()
    if brand_name_match:
        summary["Brand Name"] = brand_name_match.group(1).strip()
    if expiry_date_match:
        summary["Expiry Date"] = expiry_date_match.group(1).strip()
    if manufacturing_date_match:
        summary["Manufacturing Date"] = manufacturing_date_match.group(1).strip()
    if mrp_match:
        summary["MRP"] = mrp_match.group(1).strip()
    if price_match:
        summary["Price"] = price_match.group(1).strip()
    if quantity_match:
        summary["Quantity"] = quantity_match.group(1).strip()

    return summary

# Main function
if _name_ == "_main_":
    try:
        main()
    finally:
        if cv2.getWindowProperty('image', 0) >= 0:  # Check if a window is opened
            cv2.destroyAllWindows()